In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the HUGGINGFACE_TOKEN
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')
openai_key = os.getenv('OPENAI_KEY')

In [3]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain

# Initialize the embeddings model
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en", encode_kwargs={'normalize_embeddings': True})

# Initialize the vector store (ChromaDB in this case)
vector_store = Chroma(persist_directory='./chromaDB', collection_name="bbc_news", embedding_function=embeddings)

# Initialize the language model (OpenAI in this case)
llm = OpenAI(api_key=openai_key)

# Create the QA chain for combining documents
combine_documents_chain = load_qa_chain(llm=llm, chain_type="stuff")

# Create the RetrievalQA chain
rag_chain = RetrievalQA(
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),  # Limit to 2 documents
    combine_documents_chain=combine_documents_chain,
    return_source_documents=True
)

# Function to perform RAG-based query
def rag_query(query):
    response = rag_chain(query)
    return response

# Example usage
query = "Do you know anything about irish elections?"
response = rag_query(query)

print("Generated Response:")
print(response['result'])

print("\nSource Documents:")
for doc in response['source_documents']:
    print(doc)

Generated Response:
 I am an AI and do not have personal knowledge or experience. However, based on the context provided, the pieces of information mentioned suggest that the Irish election was a referendum on whether to ratify the EU reform treaty. It was being watched closely across Europe due to potential implications for the EU. The results were expected later on Friday and a low turnout could suggest rejection of the treaty. The treaty aimed to streamline decision-making and reduce veto powers in the EU. 

Source Documents:
page_content='Turnout during the Irish vote was being watched intently across Europe
People in the Republic of Ireland have voted in a referendum on whether to ratify the EU reform treaty.
The BBC's Jonny Dymond in Dublin says all eyes are on the turnout, as a low figure would suggest a rejection which could plunge the EU into crisis.
Some reports suggest Thursday's voter turnout was about 40%. Results are expected later on Friday.
All 27 member states have to 